In [1]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"  # 替換為你的檔案路徑
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


# 十折驗證法

In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.0 GB
Current GPU Memory Cached: 0.0 GB


In [3]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [4]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "macro_f1-score": macro_f1,
        "balanced_accuracy": balanced_acc,
        "mcc": mcc
    }

In [6]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from sklearn.model_selection import KFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [9]:
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

all_fold_results = []

# 進行 10 折交叉驗證
for fold, (train_idx, val_idx) in enumerate(kf.split(messages)):
    print(f"Fold {fold + 1}")
    
    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    print("Training set label distribution:", {label: train_labels.count(label) for label in set(train_labels)})
    print("Validation set label distribution:", {label: val_labels.count(label) for label in set(val_labels)})
        
    # 計算各 Label 的數量
    label_counts = pd.Series(train_labels).value_counts()
    avg_label_size = int(label_counts.mean())  # **取平均值作為 resample 基準點**
    print(f"avg_label_size: {avg_label_size}")
    
    # 找出需要過採樣的類別
    labels_to_oversample = label_counts[label_counts < avg_label_size].index.tolist()

    # 進行 ROS (Random Oversampling)
    ros = RandomOverSampler(sampling_strategy={label: avg_label_size for label in labels_to_oversample}, random_state=42)
    train_messages_resampled, train_labels_resampled = ros.fit_resample(np.array(train_messages).reshape(-1, 1), train_labels)
    train_messages_resampled = train_messages_resampled.flatten().tolist()

    # 轉換成 DataLoader
    train_dataset = CustomDataset(train_messages_resampled, train_labels_resampled, tokenizer, MAX_LENGTH)
    val_dataset = CustomDataset(val_messages, val_labels, tokenizer, MAX_LENGTH)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))
    model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=NUM_LABELS)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    epoch_results = []
    patience = 3
    best_macro_f1 = 0
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        epoch_results.append(metrics)
        print(metrics)

        current_macro_f1 = metrics["macro_f1-score"]
        
        if current_macro_f1 > best_macro_f1:
            best_macro_f1 = current_macro_f1  # 更新最佳 macro_f1-score
            epochs_without_improvement = 0   # 重置計數
        else:
            epochs_without_improvement += 1  # 增加計數
        
        # 若連續 patience 3 次沒有提升，則停止訓練
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    avg_metrics = {metric: np.mean([epoch[metric] for epoch in epoch_results]) for metric in epoch_results[0].keys()}
    all_fold_results.append(avg_metrics)

final_metrics = {metric: np.mean([result[metric] for result in all_fold_results]) for metric in all_fold_results[0].keys()}
print("Final 10-fold Cross-Validation Results:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

Fold 1
avg_label_size: 152
Before ROS: Counter({0: 3585, 47: 601, 17: 258, 4: 239, 48: 215, 12: 209, 11: 199, 23: 183, 42: 166, 5: 134, 2: 123, 39: 118, 18: 106, 6: 101, 26: 100, 31: 89, 22: 89, 10: 82, 49: 82, 8: 80, 9: 70, 46: 58, 25: 55, 27: 52, 19: 47, 21: 39, 32: 39, 7: 37, 33: 36, 13: 34, 36: 32, 15: 31, 34: 29, 1: 28, 3: 26, 14: 25, 37: 24, 44: 23, 16: 22, 43: 18, 30: 17, 20: 15, 24: 15, 29: 15, 28: 13, 35: 13, 45: 11, 40: 10, 41: 10, 38: 5})
After ROS: Counter({0: 3585, 47: 601, 17: 258, 4: 239, 48: 215, 12: 209, 11: 199, 23: 183, 42: 166, 1: 152, 31: 152, 39: 152, 46: 152, 2: 152, 3: 152, 5: 152, 6: 152, 7: 152, 8: 152, 9: 152, 10: 152, 14: 152, 15: 152, 16: 152, 18: 152, 19: 152, 20: 152, 21: 152, 22: 152, 24: 152, 25: 152, 26: 152, 27: 152, 49: 152, 28: 152, 29: 152, 30: 152, 32: 152, 33: 152, 34: 152, 35: 152, 36: 152, 37: 152, 13: 152, 40: 152, 41: 152, 38: 152, 43: 152, 44: 152, 45: 152})


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 743/743 [05:15<00:00,  2.35it/s]
D:\Anaconda3\envs\cuda\Lib\site-packages\sklearn\metrics\_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'val_loss': 1.478606198193892, 'val_accuracy': 0.6063829787234043, 'precision': 0.580216854017687, 'recall': 0.6063829787234043, 'f1-score': 0.5686177551034779, 'macro_f1-score': 0.307532196742514, 'balanced_accuracy': 0.37021401162515866, 'mcc': 0.47831350964181263}
Epoch 2/20


...

{'val_loss': 1.6038577534117788, 'val_accuracy': 0.6710059171597633, 'precision': 0.657945034215731, 'recall': 0.6710059171597633, 'f1-score': 0.6369008387117248, 'macro_f1-score': 0.3821808147405288, 'balanced_accuracy': 0.3850646639020755, 'mcc': 0.5708386034357626}
Epoch 6/20


100%|██████████| 744/744 [05:14<00:00,  2.37it/s]


{'val_loss': 1.4831117363470905, 'val_accuracy': 0.6674556213017752, 'precision': 0.6501680857177589, 'recall': 0.6674556213017752, 'f1-score': 0.6440192692271844, 'macro_f1-score': 0.40333531047547055, 'balanced_accuracy': 0.41538368126145475, 'mcc': 0.572191785025976}
Epoch 7/20


100%|██████████| 744/744 [05:14<00:00,  2.37it/s]


{'val_loss': 1.6469360724934992, 'val_accuracy': 0.6556213017751479, 'precision': 0.6642263624200521, 'recall': 0.6556213017751479, 'f1-score': 0.6418192723003476, 'macro_f1-score': 0.40308836118641034, 'balanced_accuracy': 0.4385563541118188, 'mcc': 0.5669851524622224}
Early stopping triggered at epoch 7
Final 10-fold Cross-Validation Results:
val_loss: 1.4284179600894533
val_accuracy: 0.6653891575024964
precision: 0.6679864123479337
recall: 0.6653891575024964
f1-score: 0.6458971334576676
macro_f1-score: 0.376694728355607
balanced_accuracy: 0.40412106145347176
mcc: 0.5536261345490706


In [9]:
all_fold_results

[{'val_loss': 1.4774756056366813,
  'val_accuracy': 0.6568880292284548,
  'precision': 0.6401314716642911,
  'recall': 0.6568880292284548,
  'f1-score': 0.6322768268287018,
  'macro_f1-score': 0.33303717513584924,
  'balanced_accuracy': 0.34832429625173766,
  'mcc': 0.5484266492879084},
 {'val_loss': 1.4332616797186701,
  'val_accuracy': 0.6721470019342359,
  'precision': 0.6488852121264229,
  'recall': 0.6721470019342359,
  'f1-score': 0.6414772608431984,
  'macro_f1-score': 0.3016922162270063,
  'balanced_accuracy': 0.30441537918176204,
  'mcc': 0.5526329841903413},
 {'val_loss': 1.3589874485534938,
  'val_accuracy': 0.6804570527974784,
  'precision': 0.6606165277994105,
  'recall': 0.6804570527974784,
  'f1-score': 0.6545120360882776,
  'macro_f1-score': 0.2908324130689872,
  'balanced_accuracy': 0.3119277966285385,
  'mcc': 0.5577025770043781},
 {'val_loss': 1.3847410363346273,
  'val_accuracy': 0.6744155503020752,
  'precision': 0.6551956633205985,
  'recall': 0.6744155503020752,
